In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
from keras.models import load_model
import cv2
import os
import numpy as np
from numpy import expand_dims
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils
import sys
import tensorflow as tf
from mtcnn import MTCNN
import time
detector = MTCNN()
# %load_ext autoreload
# %autoreload 2

np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


In [3]:
model = load_model('model/facenet_keras.h5')
print("model Loaded")
for layer in model.layers:
    layer.trianable = False  



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
model Loaded


C:\Users\Amair\Anaconda3\envs\py369\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
loaded_model = model

In [5]:
def image_to_embedding(image, model): 
    image = cv2.resize(image, (160, 160)) 
    face_pixels = image.astype('float32')
    mean, std = image.mean(), image.std()
    image = (image - mean) / std
    samples = expand_dims(image, axis=0)
    face_embedding = model.predict(samples)
    return face_embedding[0]

In [6]:
import numpy as np
from mtcnn import MTCNN
from PIL import Image
def extract_face(filename, required_size=(160, 160)):
    face_array = list()
    if  ".ipynb_checkpoints" in filename:
        print(filename)
        pass
    else:
        image = Image.open(filename)
        image = image.convert('RGB')
        pixels = np.asarray(image)
        detector = MTCNN()
        results = detector.detect_faces(pixels)
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = np.asarray(image)
    return face_array

In [7]:
def load_faces(imageDir, label_name):
    faces = list()
    faces_embedding = {}
    for filename in os.listdir(imageDir):
        path = imageDir + filename
        if ".ipynb_checkpoints" in path:
            pass
        else:
            face = extract_face(path)
            print(face.shape)
            face_emd = image_to_embedding(face, loaded_model)
            faces.append(face_emd)
    faces_embedding[label_name] = faces
    return faces, faces_embedding

In [8]:
import os
import numpy as np
def load_dataset(directory):
    X, y = list(), list()
    faces_embedding = {}
    for subdir in os.listdir(directory):
        path = directory + subdir + '/'
        if  path.startswith('.'):
            pass
        else:
            faces, f = load_faces(path, subdir)
            faces_embedding.update(f)
            label = [subdir for _ in range(len(faces))]
            print("{} length {}".format(label, len(faces)))
#             dic[label[0]] = faces
            print('>loaded %d examples for class: %s' % (len(faces), subdir))
            X.extend(faces)
            y.extend(label)
#     return X, y, faces_embedding
    return faces_embedding

In [9]:
def recognize_face(face_image, input_embeddings, model):
    embedding = image_to_embedding(face_image, loaded_model)
    minimum_distance = 200
    name = None
    for (label_name, list_of_images) in input_embeddings.items():
        for image in list_of_images:
            euclidean_distance = np.linalg.norm(embedding-image)
            print('Euclidean distance from %s is %s' %(label_name, euclidean_distance))
            if euclidean_distance < minimum_distance:
                minimum_distance = euclidean_distance
                name = label_name
    if minimum_distance <10:
        return str(name+ "     " + str(100-minimum_distance))
    else:
        return "Unknown"

In [10]:
def recognize_faces_in_cam(input_embeddings):
    cv2.namedWindow("Deklo Tum Bhi Ab")
    vc = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    while vc.isOpened():
        _, frame = vc.read()
        img = frame
        height, width, channels = frame.shape
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        identities = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
            face_image = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)]    
            identity = recognize_face(face_image, input_embeddings, loaded_model)
            if identity is not None:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(255,255,255),2)
                cv2.putText(img, str(identity), (x1+5,y1-5), font, 1, (255,255,255), 2)
        key = cv2.waitKey(100)
        cv2.imshow("Deklo Tum Bhi Ab", img)
        if key == 27: # exit on ESC
            break
    vc.release()
    cv2.destroyAllWindows()

In [19]:
input_embeddings = load_dataset("images/")


(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
(160, 160, 3)
['amair', 'amair', 'amair', 'amair'] length 4
>loaded 4 examples for class: amair
(160, 160, 3)
(160, 160, 3)
['farhan', 'farhan'] length 2
>loaded 2 examples for class: farhan
(160, 160, 3)
['srk'] length 1
>loaded 1 examples for class: srk


In [20]:
from numpy import savez_compressed
savez_compressed('C:\\Users\\Amair\\Desktop\\Project\\try-8\\model\\input-faces-embeddings.npz', input_embeddings)

In [21]:
# len(input['amair'])

In [22]:
from numpy import load
input = load('C:\\Users\\Amair\\Desktop\\Project\\try-8\\model\\input-faces-embeddings.npz')
input = input_embeddings

In [ ]:
recognize_faces_in_cam(input)

Euclidean distance from amair is 5.444157
Euclidean distance from amair is 4.993403
Euclidean distance from amair is 5.186042
Euclidean distance from amair is 5.442908
Euclidean distance from farhan is 15.1435375
Euclidean distance from farhan is 15.6764145
Euclidean distance from srk is 12.79204
Euclidean distance from amair is 8.210961
Euclidean distance from amair is 8.339287
Euclidean distance from amair is 8.148027
Euclidean distance from amair is 8.16991
Euclidean distance from farhan is 14.744134
Euclidean distance from farhan is 15.709972
Euclidean distance from srk is 12.398871
Euclidean distance from amair is 7.770452
Euclidean distance from amair is 7.64259
Euclidean distance from amair is 7.610219
Euclidean distance from amair is 7.7142687
Euclidean distance from farhan is 15.251553
Euclidean distance from farhan is 16.177166
Euclidean distance from srk is 12.894879
Euclidean distance from amair is 5.741116
Euclidean distance from amair is 5.5245595
Euclidean distance from 

In [ ]:
def newValue():
    